In [2]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
import extrair_zip_train_dir as zipService


class ImageStitchingDatasetFiles(Dataset):
    def __init__(self, folder_path, use_gradiente=False):
        self.folder = Path(folder_path)
        self.use_gradiente = use_gradiente
        # Lista todos arquivos .pt ordenados
        self.files = sorted(self.folder.glob("*.pt"))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        sample = torch.load(self.files[idx])

        def to_float_tensor(t):
            # uint8 [0..255] -> float32 [0..1]
            return t.float() / 255.0

        parte1 = to_float_tensor(sample["parte1"])
        parte2 = to_float_tensor(sample["parte2"])
        groundtruth = to_float_tensor(sample["groundtruth"])

        if self.use_gradiente:
            gradiente = to_float_tensor(sample["gradiente"])
            return (parte1, parte2), groundtruth, gradiente
        else:
            return (parte1, parte2), groundtruth

zipService.descompactar_zip_com_progresso("./train.zip", "./train")
dataset = ImageStitchingDatasetFiles("./train", use_gradiente=False)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=4, prefetch_factor=2, pin_memory=True)


⚠️ A pasta 'train' já contém arquivos. A extração foi abortada para evitar sobrescrita.


32 x 48
| Bloco                    | Altura × Largura | Canais p/ encoder | Canais Pós-concatenação |
|--------------------------|------------------|--------------------|--------------------------|
| Entrada                  | 32×48            | 3                  | —                        |
| `enc1`                   | 32×48            | 32                 | 64 (concat)              |
| `pool1`                  | 16×24            | 32                 | 64 (concat)              |
| `enc2`                   | 16×24            | 64                 | 128 (concat)             |
| `pool2`                  | 8×12             | 64                 | 128 (concat)             |
| Bottleneck (concat)      | 8×12             | —                  | 256                      |
| `dec2` entrada           | 8×12             | 256 + 128 = 384    | —                        |
| `dec2` saída             | 16×24            | 64                 | —                        |
| `dec1` entrada           | 16×24            | 64 + 64 = 128      | —                        |
| `dec1` saída             | 32×48            | 32                 | —                        |
| Saída final              | 32×48            | 3                  | —                        |

64x96
| Bloco                    | Altura × Largura | Canais p/ encoder | Canais Pós-concatenação |
|--------------------------|------------------|--------------------|--------------------------|
| Entrada                  | 64×96            | 3                  | —                        |
| `enc1`                   | 64×96            | 32                 | 64 (concat)              |
| `pool1`                  | 32×48            | 32                 | 64 (concat)              |
| `enc2`                   | 32×48            | 64                 | 128 (concat)             |
| `pool2`                  | 16×24            | 64                 | 128 (concat)             |
| Bottleneck (concat)      | 16×24            | —                  | 256                      |
| `dec2` entrada           | 16×24            | 256 + 128 = 384    | —                        |
| `dec2` saída             | 32×48            | 64                 | —                        |
| `dec1` entrada           | 32×48            | 64 + 64 = 128      | —                        |
| `dec1` saída             | 64×96            | 32                 | —                        |
| Saída final              | 64×96            | 3                  | —                        |


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# CBAM (Convolutional Block Attention Module)
# Aplica atenção canal + espacial separadamente
class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAM, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1, bias=False)
        )

        self.sigmoid_channel = nn.Sigmoid()
        self.conv_spatial = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)
        self.sigmoid_spatial = nn.Sigmoid()

    def forward(self, x):
        # Atenção no canal
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        x_out = x * self.sigmoid_channel(avg_out + max_out)  # salva num novo tensor para não perder o input original

        # Atenção espacial
        avg_out = torch.mean(x_out, dim=1, keepdim=True)
        max_out, _ = torch.max(x_out, dim=1, keepdim=True)
        spatial_attention = torch.cat([avg_out, max_out], dim=1)  # [N, 2, H, W]
        spatial_attention = self.sigmoid_spatial(self.conv_spatial(spatial_attention))  # [N, 1, H, W]

        # Multiplica o resultado da atenção espacial pelo tensor original (com canais corretos)
        out = x_out * spatial_attention

        return out

# Self-Attention simples no bottleneck
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Conv2d(in_dim, in_dim // 8, 1)
        self.key = nn.Conv2d(in_dim, in_dim // 8, 1)
        self.value = nn.Conv2d(in_dim, in_dim, 1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        B, C, H, W = x.size()
        proj_query = self.query(x).view(B, -1, H * W).permute(0, 2, 1)
        proj_key = self.key(x).view(B, -1, H * W)
        energy = torch.bmm(proj_query, proj_key)  # matriz de atenção
        attention = F.softmax(energy, dim=-1)

        proj_value = self.value(x).view(B, -1, H * W)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(B, C, H, W)
        return self.gamma * out + x

# Bloco de codificação padrão
class EncoderBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

# Bloco de decodificação com upsample + concat + convoluções
class DecoderBlock(nn.Module):
    def __init__(self, ch_in, ch_skip, ch_out):
        super().__init__()
        self.up = nn.ConvTranspose2d(ch_in, ch_out, kernel_size=2, stride=2)
        self.conv = nn.Sequential(
            nn.Conv2d(ch_out + ch_skip, ch_out, 3, padding=1),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch_out, ch_out, 3, padding=1),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
        )

    def forward(self, x, skip):
        x = self.up(x)
        if x.shape[2:] != skip.shape[2:]:
            x = F.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        return x

# Rede UNet com dois encoders, CBAM e self-attention no bottleneck
class DualEncoderUNet_CBAM_SA_Small(nn.Module):
    def __init__(self, in_channels=3, base_ch=32):
        super().__init__()

        # Dois encoders independentes (parte1 e parte2)
        self.enc1_1 = EncoderBlock(in_channels, base_ch)
        self.enc2_1 = EncoderBlock(base_ch, base_ch * 2)

        self.enc1_2 = EncoderBlock(in_channels, base_ch)
        self.enc2_2 = EncoderBlock(base_ch, base_ch * 2)

        self.pool = nn.MaxPool2d(2)

        # Bottleneck com self-attention
        self.bottleneck = EncoderBlock(base_ch * 4, base_ch * 4)
        self.attn = SelfAttention(base_ch * 4)

        # CBAM nas skip connections
        self.cbam2 = CBAM(base_ch * 4)
        self.cbam1 = CBAM(base_ch * 2)

        # Decoder com três parâmetros por bloco
        self.dec2 = DecoderBlock(base_ch * 4, base_ch * 4, base_ch * 2)  # 128, 128, 64
        self.dec1 = DecoderBlock(base_ch * 2, base_ch * 2, base_ch)      # 64, 64, 32

        self.final = nn.Conv2d(base_ch, 3, kernel_size=1)

    def forward(self, x1, x2):
        # Encoder para parte1
        e1_1 = self.enc1_1(x1)
        e2_1 = self.enc2_1(self.pool(e1_1))

        # Encoder para parte2
        e1_2 = self.enc1_2(x2)
        e2_2 = self.enc2_2(self.pool(e1_2))

        # Garantir que as features estejam com mesmas dimensões (por segurança)
        if e1_1.shape[2:] != e1_2.shape[2:]:
            e1_2 = F.interpolate(e1_2, size=e1_1.shape[2:], mode='bilinear', align_corners=False)
        if e2_1.shape[2:] != e2_2.shape[2:]:
            e2_2 = F.interpolate(e2_2, size=e2_1.shape[2:], mode='bilinear', align_corners=False)

        # Bottleneck: concatenação + atenção
        b = self.bottleneck(torch.cat([self.pool(e2_1), self.pool(e2_2)], dim=1))
        b = self.attn(b)

        if debug > 0: print("b shape:", b.shape)
        if debug > 0: print("skip2 shape:", self.cbam2(torch.cat([e2_1, e2_2], dim=1)).shape)
        if debug > 0: print("skip1 shape:", self.cbam1(torch.cat([e1_1, e1_2], dim=1)).shape)


        # Decoder com CBAM nas skip connections
        d2 = self.dec2(b, self.cbam2(torch.cat([e2_1, e2_2], dim=1)))
        d1 = self.dec1(d2, self.cbam1(torch.cat([e1_1, e1_2], dim=1)))

        return torch.sigmoid(self.final(d1))  # saída com valo
import torch.nn as nn

class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=9):  # Agora espera parte1 (3) + parte2 (3) + target/fake (3)
        super(PatchDiscriminator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(in_channels, 64, normalize=False),  # in_channels = 9
            *block(64, 128),
            *block(128, 256),
            *block(256, 512),
            nn.Conv2d(512, 1, kernel_size=4, padding=1)  # saída do PatchGAN (mapa de decisão)
        )

    def forward(self, img):
        return self.model(img)

In [8]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from pytorch_msssim import ssim, ms_ssim
import lpips
from metrics import compute_all_metrics  # Importa a função de métricas do arquivo metrics.py
from tqdm import tqdm

debug = 0
# from generator import DualEncoderUNet_CBAM_SA_Small  # novo gerador com 2 encoders, CBAM e SelfAttention
# from discriminator import PatchDiscriminator  # ou caminho equivalente

# LPIPS usa um modelo de rede para comparação perceptual
lpips_fn = lpips.LPIPS(net='alex')

def carregar_checkpoint_mais_recente(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint_epoch") and f.endswith(".pt")]
    if not checkpoints:
        return None

    checkpoints.sort(key=lambda x: int(x.split("epoch")[1].split(".")[0]))
    return os.path.join(checkpoint_dir, checkpoints[-1])


def train(
    generator, discriminator, dataloader, device, epochs,
    save_every, checkpoint_dir, checkpoint_batch_dir,
    tensorboard_dir, metrics, lr_g=2e-4, lr_d=2e-4,
    lr_min=1e-6, gen_steps_per_batch=1
):
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(checkpoint_batch_dir, exist_ok=True)
    writer = SummaryWriter(tensorboard_dir)

    criterion_GAN = nn.BCEWithLogitsLoss()
    criterion_L1 = nn.L1Loss()

    optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))

    scheduler_G = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_G, T_max=epochs, eta_min=lr_min)
    scheduler_D = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_D, T_max=epochs, eta_min=lr_min)

    start_epoch = 0
    checkpoint_path = carregar_checkpoint_mais_recente(checkpoint_dir)
    if checkpoint_path:
        print(f"🔁 Carregando checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        generator.load_state_dict(checkpoint["generator_state_dict"])
        discriminator.load_state_dict(checkpoint["discriminator_state_dict"])
        optimizer_G.load_state_dict(checkpoint["optimizer_G_state_dict"])
        optimizer_D.load_state_dict(checkpoint["optimizer_D_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        print(f"✔️ Retomando a partir da época {start_epoch}")

    last_checkpoint_time = time.time()

    for epoch in range(start_epoch, epochs):
        pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}/{epochs}")
        for i, ((part1, part2), target) in pbar:            
            part1 = part1.to(device)
            part2 = part2.to(device)
            target = target.to(device)

            real_input = torch.cat([part1, part2, target], dim=1)
            fake = generator(part1, part2)
            fake_input = torch.cat([part1, part2, fake.detach()], dim=1)

            # Train Discriminator
            optimizer_D.zero_grad()
            pred_real = discriminator(real_input)
            pred_fake = discriminator(fake_input)

            loss_D_real = criterion_GAN(pred_real, torch.ones_like(pred_real))
            loss_D_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))
            loss_D = (loss_D_real + loss_D_fake) / 2
            loss_D.backward()
            optimizer_D.step()

            # Train Generator
            for _ in range(gen_steps_per_batch):
                fake = generator(part1, part2)
                fake_input = torch.cat([part1, part2, fake], dim=1)
                optimizer_G.zero_grad()
                pred_fake = discriminator(fake_input)
                loss_G_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake))
                loss_G_L1 = criterion_L1(fake, target)
                loss_G = 8.0 * loss_G_GAN + 2.0 * loss_G_L1
                loss_G.backward()
                optimizer_G.step()

            pbar.set_postfix({
                "loss_G": f"{loss_G.item():.4f}",
                "loss_D": f"{loss_D.item():.4f}"
            })

            writer.add_scalar("Loss/Generator", loss_G.item(), epoch * len(dataloader) + i)
            writer.add_scalar("Loss/Discriminator", loss_D.item(), epoch * len(dataloader) + i)

            with torch.no_grad():
                eval_metrics = compute_all_metrics(fake, target, part1, part2, writer, epoch * len(dataloader) + i)
                for k, v in eval_metrics.items():
                    if v is not None:
                        writer.add_scalar(f"Metrics/{k}", v, epoch * len(dataloader) + i)

            # Checkpoint a cada 10 minutos
            if time.time() - last_checkpoint_time > 600:
                torch.save({
                    'epoch': epoch,
                    'batch': i,
                    'generator_state_dict': generator.state_dict(),
                    'discriminator_state_dict': discriminator.state_dict(),
                    'optimizer_G_state_dict': optimizer_G.state_dict(),
                    'optimizer_D_state_dict': optimizer_D.state_dict(),
                }, os.path.join(checkpoint_batch_dir, f'checkpoint_epoch{epoch}_batch{i}.pt'))
                last_checkpoint_time = time.time()

        # Fim da época: salvar checkpoint principal
        torch.save({
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'optimizer_G_state_dict': optimizer_G.state_dict(),
            'optimizer_D_state_dict': optimizer_D.state_dict(),
        }, os.path.join(checkpoint_dir, f'checkpoint_epoch{epoch}.pt'))

        scheduler_G.step()
        scheduler_D.step()

    writer.close()


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/prkd/anaconda3/envs/ImageStitching/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/prkd/anaconda3/envs/ImageStitching/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/prkd/anaconda3/envs/ImageStitching/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth


/home/prkd/anaconda3/envs/ImageStitching/lib/python3.12/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="torch")


# Montar o caminho do checkpoint e logs para o tensorboard
!sshfs prkdvps@64.71.153.122:/home/prkdvps/tensorboard/logs /home/prkd/gan-image-stitching-training/gan_image_stitching_training/logs
!sshfs prkdvps@64.71.153.122:/home/prkdvps/tensorboard/checkpoints_epoch /home/prkd/gan-image-stitching-training/gan_image_stitching_training/checkpoints_epoch/
!sshfs prkdvps@64.71.153.122:/home/prkdvps/tensorboard/checkpoints_batch/ /home/prkd/gan-image-stitching-training/gan_image_stitching_training/checkpoints_batch/


# Hiperparâmetros
num_epochs = 100
gen_steps_per_batch = 20
learning_rate = 2e-4
lr_min = 1e-5
lr_max = 2e-4
log_interval = 600  # em segundos (10 minutos)

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exemplo de chamada (fora do train.py):
generator = DualEncoderUNet_CBAM_SA_Small()
discriminator = PatchDiscriminator()
train(generator, discriminator, dataloader, device, epochs=200, save_every=600,
      checkpoint_dir="./checkpoints_epoch", checkpoint_batch_dir="./checkpoints_batch",
      tensorboard_dir="./logs/32x48", metrics=True, gen_steps_per_batch=20)



Epoch 1/200:   0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_353957/2836379704.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sample = torch.load(self.files[idx])


KeyboardInterrupt: 

In [ ]:
import torch
import matplotlib.pyplot as plt

import torch
import matplotlib.pyplot as plt

def visualizar_amostra_pt(caminho_pt):
    """
    Visualiza a amostra salva no arquivo .pt no formato esperado:
    dicionário com chaves: 'parte1', 'parte2', 'mask', 'groundtruth', 'gradiente'.
    Cada tensor é uint8, shape [C, H, W].

    Parâmetros:
        caminho_pt (str ou Path): caminho do arquivo .pt a ser aberto
    """
    sample = torch.load(caminho_pt)

    print("Chaves no arquivo:", list(sample.keys()))
    for k, v in sample.items():
        print(f"{k}: shape {v.shape}, dtype {v.dtype}")

    # Converter para formato H x W x C e mostrar com matplotlib
    def tensor_to_img(tensor):
        # tensor [C, H, W], uint8
        img = tensor.permute(1, 2, 0).cpu().numpy()
        return img

    plt.figure(figsize=(15, 8))

    for i, key in enumerate(['parte1', 'parte2', 'mask', 'groundtruth', 'gradiente'], 1):
        if key in sample:
            img = tensor_to_img(sample[key])
            shape_str = sample[key].shape
            plt.subplot(2, 3, i)
            plt.imshow(img)
            plt.title(f"{key} - {shape_str}")

    plt.tight_layout()
    plt.show()

visualizar_amostra_pt("./train/000000009286_sample10.pt")
